# 스테이블 디퓨전 XL 터보

SDXL 터보는 적대적 시간 증류 [스테이블 디퓨전 XL](https://huggingface.co/papers/2307.01952) (SDXL) 모델로,
단 1단계 만에 추론을 실행할 수 있습니다.

이 가이드에서는 텍스트-이미지 변환 및 이미지-이미지 변환에 SDXL-터보를 사용하는 방법을 보여줍니다.

시작하기 전에 다음 라이브러리가 설치되어 있는지 확인하십시오.

In [ ]:
# Colab에서 필요한 라이브러리를 설치하려면 주석 처리를 해제하세요.
#!pip install -q diffusers transformers accelerate

## 모델 체크포인트 로드

모델 가중치는 허브 또는 로컬의 별도 하위 폴더에 저장될 수 있으며, 이 경우 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드를 사용해야 합니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipeline = pipeline.to("cuda")

또한 [from_single_file()](https://huggingface.co/docs/diffusers/main/en/api/loaders/single_file#diffusers.loaders.FromSingleFileMixin.from_single_file) 메서드를 사용하여 허브 또는 로컬에서 단일 파일 형식(`.ckpt` 또는 `.safetensors`)으로 저장된 모델 체크포인트를 로드할 수도 있습니다. 이 로드 방법의 경우 `timestep_spacing="trailing"`으로 설정해야 합니다(더 나은 결과를 얻기 위해 다른 스케줄러 구성 값을 자유롭게 실험해 보세요).

In [ ]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler
import torch

pipeline = StableDiffusionXLPipeline.from_single_file(
    "https://huggingface.co/stabilityai/sdxl-turbo/blob/main/sd_xl_turbo_1.0_fp16.safetensors",
    torch_dtype=torch.float16, variant="fp16")
pipeline = pipeline.to("cuda")
pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeline.scheduler.config, timestep_spacing="trailing")

## 텍스트-이미지 변환

텍스트-이미지 변환의 경우 텍스트 프롬프트를 전달합니다. 기본적으로 SDXL 터보는 512x512 이미지를 생성하며 이 해상도에서 최상의 결과를 얻을 수 있습니다. `height` 및 `width` 매개변수를 768x768 또는 1024x1024로 설정해 볼 수 있지만 이렇게 하면 품질이 저하될 수 있습니다.

모델이 이 매개변수 없이 훈련되었으므로 `guidance_scale`을 0.0으로 설정하여 비활성화해야 합니다. 단일 추론 단계만으로도 고품질 이미지를 생성하기에 충분합니다.
단계 수를 2, 3 또는 4로 늘리면 이미지 품질이 향상됩니다.

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipeline_text2image = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipeline_text2image = pipeline_text2image.to("cuda")

prompt = "복잡한 이탈리아 사제복을 입은 아기 너구리의 영화 같은 장면."

image = pipeline_text2image(prompt=prompt, guidance_scale=0.0, num_inference_steps=1).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/sdxl-turbo-text2img.png" alt="로브를 입은 너구리의 생성된 이미지"/>
</div>

## 이미지-이미지 변환

이미지-이미지 변환의 경우 `num_inference_steps * strength`가 1보다 크거나 같은지 확인합니다.
이미지-이미지 파이프라인은 `int(num_inference_steps * strength)` 단계 동안 실행됩니다. 예를 들어
아래 예에서는 `0.5 * 2.0 = 1` 단계입니다.

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

# 체크포인트를 로드할 때 추가 메모리를 소비하지 않도록 from_pipe를 사용합니다.
pipeline_image2image = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat.png")
init_image = init_image.resize((512, 512))

prompt = "고양이 마법사, 간달프, 반지의 제왕, 디테일, 판타지, 귀여움, 사랑스러움, 픽사, 디즈니, 8k"

image = pipeline_image2image(prompt, image=init_image, strength=0.5, guidance_scale=0.0, num_inference_steps=2).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/sdxl-turbo-img2img.png" alt="SDXL 터보를 사용한 이미지-이미지 변환 샘플"/>
</div>

## SDXL 터보 속도 더욱 향상시키기

- PyTorch 버전 2.0 이상을 사용하는 경우 UNet을 컴파일합니다. 첫 번째 추론 실행은 매우 느리지만 이후 실행은 훨씬 빨라집니다.

In [ ]:
pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

- 기본 VAE를 사용하는 경우 각 생성 전후에 비용이 많이 드는 `dtype` 변환을 피하기 위해 `float32`로 유지합니다. 첫 번째 생성 전에 이 작업을 한 번만 수행하면 됩니다.

In [ ]:
pipe.upcast_vae()

대안으로 커뮤니티 회원 [`@madebyollin`](https://huggingface.co/madebyollin)이 만든 [16비트 VAE](https://huggingface.co/madebyollin/sdxl-vae-fp16-fix)를 사용할 수도 있습니다. 이 VAE는 `float32`로 업캐스트할 필요가 없습니다.